In [501]:
import pandas as pd
import json
import numpy as np
import ast

In [550]:
train = pd.read_json('./origin_data_modify/train_mod5.json', typ='frame')
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80953 entries, 0 to 80952
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          80953 non-null  object
 1   id            80953 non-null  int64 
 2   plylst_title  80953 non-null  object
 3   songs         80953 non-null  object
 4   like_cnt      80953 non-null  int64 
 5   updt_date     80953 non-null  object
dtypes: int64(2), object(4)
memory usage: 4.3+ MB


- 태그 별 좋아요 수를 기준으로 sorting

In [551]:
train['tags'] = train['tags'].astype(str)
train['tags'][0][0]

'['

In [552]:
train[train['tags'] == "['']"]

,tags,id,plylst_title,songs,like_cnt,updt_date
5314,[''],58486,한국인듷,"[588648, 130871, 202564, 695590, 144582, 21015...",7,2017-10-11 09:19:27.000
20246,[''],26122,입문입문,"[580332, 10391, 223964, 144582, 256571, 232762...",5,2017-10-11 09:19:54.000
32338,[''],106238,아앙ㅇㅜㅠ,"[140638, 515089, 453750, 704374, 434027, 35001...",5,2017-10-11 09:16:58.000
51868,[''],52003,뚜두두둗ㄴ,"[57266, 332854, 99218, 44347, 7434, 520617, 27...",5,2017-10-11 09:17:55.000
51928,[''],74509,떨어진다아아,"[271384, 662434, 533893, 104230, 634553, 35038...",9,2017-10-11 09:18:17.000
69052,[''],118037,간지러워,"[434371, 288842, 184408, 662486, 233607, 55624...",5,2017-10-11 09:18:44.000


In [553]:
80953-6

80947

In [554]:
# tag 값을 갖고 있지 않는 플레이리스트 버리기
train = train[train['tags'] != "['']"]
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80947 entries, 0 to 80952
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          80947 non-null  object
 1   id            80947 non-null  int64 
 2   plylst_title  80947 non-null  object
 3   songs         80947 non-null  object
 4   like_cnt      80947 non-null  int64 
 5   updt_date     80947 non-null  object
dtypes: int64(2), object(4)
memory usage: 4.3+ MB


In [555]:
# 데이터프레임을 사용할 때, apply 함수를 사용하여 'tags' 열에 대해 변환을 적용
train['tags'] = train['tags'].apply(lambda x: ast.literal_eval(x))

In [556]:
train_copy = train.copy()
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80947 entries, 0 to 80952
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          80947 non-null  object
 1   id            80947 non-null  int64 
 2   plylst_title  80947 non-null  object
 3   songs         80947 non-null  object
 4   like_cnt      80947 non-null  int64 
 5   updt_date     80947 non-null  object
dtypes: int64(2), object(4)
memory usage: 4.3+ MB


In [557]:
# 빈 태그 값을 갖고 있는 플레이리스트 추출

ply_id_del = [4435, 50403, 82487, 110957]
train_copy[train_copy['id'].isin(ply_id_del)]

,tags,id,plylst_title,songs,like_cnt,updt_date
54265,"[, 두근두근, 할수있을까, 과연, 올해는, 연애, 설레는]",4435,두근두근ㅇ_ㅇ 연애하고 싶게 만드는 달콤쏭,"[101572, 339005, 41567, 185929, 37965, 241866,...",7,2017-02-17 12:01:47.000
66170,"[힙합, Rap, 시즌2, 랩, HipHop, , 래퍼, 고등래퍼, 시즌1]",50403,고등래퍼 시즌1 참가자들의 방송 음원과 주요 발표곡들,"[658937, 204018, 616883, 399249, 27861, 6563, ...",47,2020-01-21 11:11:11.000
66804,"[겨울, 분위기, 매장음악, 카페, 연주곡, 잔잔한, 추억, 힐링, 일상, , 까페]",82487,겨울에 끝에서 듣는 힐링곡,"[106759, 151902, 655967, 138635, 393534, 67895...",4,2017-02-06 16:44:07.000
70884,"[, 가사로듣는, 영화]",110957,가사로 듣는 영화 17화 ㅋㅋ,"[612189, 219740, 569548, 471185, 188789, 19863...",2,2016-07-08 13:01:43.000


In [559]:
train_copy.at[54265, 'tags'] = ['두근두근', '할수있을까', '과연', '올해는', '연애', '설레는']
train_copy.at[66170, 'tags'] = ['힙합', 'Rap', '시즌2', '랩', 'HipHop', '래퍼', '고등래퍼', '시즌1']	
train_copy.at[66804, 'tags'] = 	['겨울', '분위기', '매장음악', '카페', '연주곡', '잔잔한', '추억', '힐링', '일상', '까페']
train_copy.at[70884, 'tags'] = ['가사로듣는', '영화']	

In [560]:
# 확인
train_copy[train_copy['id'].isin(ply_id_del)]

,tags,id,plylst_title,songs,like_cnt,updt_date
54265,"[두근두근, 할수있을까, 과연, 올해는, 연애, 설레는]",4435,두근두근ㅇ_ㅇ 연애하고 싶게 만드는 달콤쏭,"[101572, 339005, 41567, 185929, 37965, 241866,...",7,2017-02-17 12:01:47.000
66170,"[힙합, Rap, 시즌2, 랩, HipHop, 래퍼, 고등래퍼, 시즌1]",50403,고등래퍼 시즌1 참가자들의 방송 음원과 주요 발표곡들,"[658937, 204018, 616883, 399249, 27861, 6563, ...",47,2020-01-21 11:11:11.000
66804,"[겨울, 분위기, 매장음악, 카페, 연주곡, 잔잔한, 추억, 힐링, 일상, 까페]",82487,겨울에 끝에서 듣는 힐링곡,"[106759, 151902, 655967, 138635, 393534, 67895...",4,2017-02-06 16:44:07.000
70884,"[가사로듣는, 영화]",110957,가사로 듣는 영화 17화 ㅋㅋ,"[612189, 219740, 569548, 471185, 188789, 19863...",2,2016-07-08 13:01:43.000


In [561]:
tag_like_cnt = train_copy.loc[:, ['tags', 'like_cnt', 'plylst_title', 'id']]
tag_like_cnt.head()

,tags,like_cnt,plylst_title,id
0,[팝],5,1 HITS OF DECADE SPECIAL 2,50564
1,"[기분좋은, 100번]",4,100번 들어도 기분좋은 가요,148495
2,"[겨울, 크리스마스]",353,12월의 재즈,117162
3,[발라드],29,12월의 크리스마스를 yeah,151638
4,[일렉],84,1월의 최신 EDM 소식,27984


In [562]:
tag_explode = tag_like_cnt.explode('tags').reset_index(drop=True)
tag_explode

,tags,like_cnt,plylst_title,id
0,팝,5,1 HITS OF DECADE SPECIAL 2,50564
1,기분좋은,4,100번 들어도 기분좋은 가요,148495
2,100번,4,100번 들어도 기분좋은 가요,148495
3,겨울,353,12월의 재즈,117162
4,크리스마스,353,12월의 재즈,117162
...,...,...,...,...
350274,비오는날,6,비오는날 듣기 좋은 연주곡 모음,106660
350275,추억,6,비오는날 듣기 좋은 연주곡 모음,106660
350276,회상,6,비오는날 듣기 좋은 연주곡 모음,106660
350277,비오는날,12,비 오는 날 어울리는 몽환적인 힐링 피아노 모음,150419


In [563]:
# 'tags' 열을 기준으로 그룹화하고 'like_cnt' 합하고 'plylst_title'을 새로운 리스트에 담음
tag_group = tag_explode.groupby('tags').agg({'like_cnt': 'sum', 'plylst_title': list, 'id': list}).reset_index()
tag_group

,tags,like_cnt,plylst_title,id
0,00,158,"[추억소환 신나는 댄스음악, Now and then 레트로 팝, I LOVE 20...","[38363, 81830, 123921]"
1,007,69,"[긴장감 넘치는 액션과 사운드 첩보 스파이물 대표 OST, 007 제임스 본드 테...","[89985, 45506]"
2,007시리즈,65,"[클래식한 스파이 액션의 대표주자 007 시리즈를 빛낸 역대 주제곡, 당대 최고 ...","[12845, 139999]"
3,00s,10,[00년대 많이 듣던 발라드 모음],[46831]
4,00년,30,"[미안함 사랑 그리움 다 느껴져 술 땡기는 밤듣기 좋은 발라드, 국내차트 올킬...","[53271, 68763, 67733, 112447]"
...,...,...,...,...
24442,힙해,2367,"[힙해 편집샵st 플레이리스트, 요즘 가장 핫한 2인조 프로듀싱팀 그루비룸의 음...","[40247, 119138, 152409, 76132, 71426]"
24443,힙힙힙,19,"[달달힙한노래 모음, 겨울에도 힙하게 얼죽힙 뉴트로한 뮤지션 모음집]","[59850, 45778]"
24444,힛뎀포크,41,[요즘 힙클 대세 Hit Them Folks를 아시나요],[37691]
24445,힛뎀폭,41,[요즘 힙클 대세 Hit Them Folks를 아시나요],[37691]


In [564]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [566]:
# 범주형 변수를 원핫 인코딩
data_encoded = pd.get_dummies(tag_group['tags'], columns=['Category'])
data_encoded

,00,007,007시리즈,00s,00년,00년대,00년대노래,00년대발라드,00년도노래,00년생,...,힙합팬이창민,힙합페스티벌,힙합플레이리스트,힙합플레이야,힙합플레이야2018,힙해,힙힙힙,힛뎀포크,힛뎀폭,힛뎀폭스
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
24443,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
24444,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
24445,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [567]:
# 특성과 타겟 데이터 분리

X = data_encoded[1000:]  # 특성
y = data_encoded[:1000]  # 타겟

In [568]:
# 데이터 분할 (필요에 따라)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [23447, 1000]

In [23]:
knn = KNeighborsClassifier()

In [24]:
# 모델 훈련
knn.fit(X, y)

# 예측
y_pred = knn.predict(X_test)

ValueError: Found input variables with inconsistent numbers of samples: [23448, 1000]

In [ ]:
kn.fit(tags_raw, target)

ValueError: could not convert string to float: '벽난로'